In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

import re
import time

In [ ]:
## Project title

In [ ]:
## Business use case

In [ ]:
## Define problem statement

In [ ]:

url = 'https://www.makaan.com/hyderabad-residential-property/rent-property-in-hyderabad-city?page=1'


In [ ]:
for i in range(1,51):
    url = f'https://www.makaan.com/hyderabad-residential-property/rent-property-in-hyderabad-city?page={i}'
    print(url)

In [ ]:
page=requests.get(url)

In [ ]:
page.status_code

In [ ]:
page.text

In [ ]:
soup = BeautifulSoup(page.text)

In [ ]:
a = soup.find_all('div',attrs={'class':'infoWrap'})

In [ ]:
a[0].text

In [ ]:
for i in soup.find_all('div',attrs={'class':'infoWrap'}):
    price = i.find('td',attrs={'class':'price'})
    print(price.text)

In [ ]:
for i in soup.find_all('div',attrs={'class':'infoWrap'}):
    area = i.find('td',attrs={'class':'size'})
    print(area.text)

In [ ]:
for i in soup.find_all('div',attrs={'class':'infoWrap'}):
    furniture = i.find('td',attrs={'class':'val'})
    print(furniture.text)

In [ ]:
for i in soup.find_all('div',attrs={'class':'infoWrap'}):
    bhk = i.find('div',attrs={'class':'title-line'})
    print(bhk.text)

In [ ]:
for i in soup.find_all('div',attrs={'class':'infoWrap'}):
    loc = i.find('span',attrs={'class':'locName'})
    print(loc.text)

In [ ]:
for i in soup.find_all('div',attrs={'class':'infoWrap'}):
    bathroom = i.find('ul',attrs={'class':'listing-details'})
    print(bathroom.text)

In [ ]:
for i in soup.find_all('div',attrs={'class':'infoWrap'}):
    phone = i.find('div',attrs={'class':'callwrap'})
    print(phone.text)

## Extracting Data from Website

In [ ]:
price=[]
sq_area=[]
furniture=[]
bhk=[]
location=[]
bathroom=[]
pagenum=[]
for i in range(1,51):
    url = f'https://www.makaan.com/hyderabad-residential-property/rent-property-in-hyderabad-city?page={i}'
    page=requests.get(url)
    soup=BeautifulSoup(page.text)
    
    for x in soup.find_all('div',attrs={'class':'infoWrap'}):
        cost = x.find('td',attrs={'class':'price'})
        area = x.find('td',attrs={'class':'size'})
        furni = x.find('td',attrs={'class':'val'})
        Bhk = x.find('div',attrs={'class':'title-line'})
        loc = x.find('span',attrs={'class':'locName'})
        bath = x.find('ul',attrs={'class':'listing-details'})
        
        if cost is None:
            price.append(np.NaN)
        else:
            price.append(cost.text)
        
        if area is None:
            sq_area.append(np.NaN)
        else:
            sq_area.append(area.text)
        
        if furni is None:
            furniture.append(np.NaN)
        else:
            furniture.append(furni.text)
            
        if Bhk is None:
            bhk.append(np.NaN)
        else:
            bhk.append(Bhk.text)
        
        if loc is None:
            location.append(np.NaN)
        else:
            location.append(loc.text)
        
        if bath is None:
            bathroom.append(np.NaN)
        else:
            bathroom.append(bath.text)
        
        pagenum.append(i)

In [ ]:
print(len(price))
print(len(sq_area))
print(len(furniture))
print(len(bhk))
print(len(location))
print(len(bathroom))

In [ ]:
house_df=pd.DataFrame({'price':price,'sq_area':sq_area,'furinture':furniture,'BHK':bhk,'location':location,'bathrooms':bathroom})

In [ ]:
house_df.head()

In [ ]:
house_df.to_csv('House_data.csv')

In [ ]:
house_df['location'].value_counts()

#### Reading CSV

In [ ]:
df = pd.read_csv('House_data.csv')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.dtypes

In [ ]:
df.describe(include="all")

In [ ]:
df.head()

#### Removing "," and 'L' from price column

In [ ]:
def price_mod(x):
    for i in x:
        if "," in x:
            return x.replace(",","")
        elif "L" in x:
            return float(x.replace(" L",""))*100000

In [ ]:
df['Price_in_rupees']=df['price'].apply(price_mod)

In [ ]:
df.head()

In [ ]:
df.dtypes

#### changing type of Price_in_rupees from object to int64


In [ ]:
df['Price_in_rupees']=df['Price_in_rupees'].astype(np.int64)

In [ ]:
df.dtypes

#### Seperating digits from BHK

In [ ]:
df['BHK_']=df['BHK'].apply(lambda x:x[0])

In [ ]:
df['BHK_'].isnull().sum()

In [ ]:
df['BHK_'].value_counts()

#### Seperating House_type from BHK column

In [ ]:
df["House_Type"]=df['BHK'].apply(lambda x: re.findall(r"(Apartment|Independent House|Independent Floor|Studio Apartment|Villa)",x)[0])

In [ ]:
df["House_Type"].value_counts()

In [ ]:
df["House_Type"].isnull().sum()

In [ ]:
df.head()

### Spliting city from location

In [ ]:
df['Location']=df['location'].apply(lambda x:x.split(',')[1])

In [ ]:
df.head()

In [ ]:
df['Location']

#### Seperating sub_location from location column

In [ ]:
df['Locality']=df['location'].apply(lambda x:x.split(',')[0])

In [ ]:
df.head()

In [ ]:
df['Locality'].isnull().sum()

In [ ]:
df['Locality'].value_counts()

#### Seperating No_0f_Bathrooms from bathrooms column using Regex

In [ ]:
def regx_1(x):
    
    for i in x:
        if re.findall(r'[0-9]',x)!=[]:
            return re.findall(r'[0-9]',x)[0]
        else:
            return float("NaN")
            

In [ ]:
df['No_of_Bathrooms']=df['bathrooms'].apply(regx_1)

In [ ]:
df.head()

In [ ]:
df['No_of_Bathrooms'].isnull().sum()

In [ ]:
df['No_of_Bathrooms'].value_counts()

#### Seperating Facing using Regex

In [ ]:
def regx_2(x):
    
    for i in x:
        if re.findall(r"[EWNS][\w]+\s[a-z]+",x)!=[]:
            return re.findall(r"[EWNS][\w]+\s[a-z]+",x)[0]
        else:
            return float("NaN")
            

In [ ]:
df["Facing"]=df['bathrooms'].apply(regx_2)

In [ ]:
df.head()

In [ ]:
df["Facing"].value_counts()

In [ ]:
df["Facing"].isnull().sum()

In [ ]:
df.tail()

#### dropping unwanted columns

In [ ]:
df.drop(['BHK','price','bathrooms','Unnamed: 0','location'],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df.to_csv('House_data_arranged.csv')

#### Checking for duplicates

In [ ]:
df.duplicated().sum()

#### Reading arranged table

In [ ]:
df1 = pd.read_csv('House_data_arranged.csv')

In [ ]:
df1.head()

In [ ]:
df1.drop(['Unnamed: 0'],axis =1,inplace=True)

In [ ]:
df1.head()

In [ ]:
df1.info()

## Cleaning Data

#### checking for duplicates

In [ ]:
df1.duplicated().sum()

#### Dropping Duplicates

In [ ]:
df1.drop_duplicates(inplace=True)

In [ ]:
df1.shape

#### checking for null values

In [ ]:
df1.isna().sum()

In [ ]:
df1['Facing'].value_counts()

In [ ]:
df1.isnull().sum()/df1.shape[0]

In [ ]:
df1.groupby('Locality')['Facing'].value_counts()

In [ ]:
df1.head()

#### Dropping Null values

In [ ]:
df1.dropna(subset=['Facing'],inplace=True)

In [ ]:
df1.head()

In [ ]:
df1.isnull().sum()/df1.shape[0]

In [ ]:
df1.reset_index(inplace=True)

In [ ]:
df1.head()

#### Dropping Location and index columns

In [ ]:
df1.drop(['Location','index'],axis=1,inplace=True)

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.duplicated().sum()

In [ ]:
df1.isnull().sum()

In [ ]:
df1.head()

In [ ]:
df1.info()

In [ ]:
df1[df1['No_of_Bathrooms']==np.NaN]

In [ ]:
df1['No_of_Bathrooms']=df1['No_of_Bathrooms'].astype(np.int64)

In [ ]:
df1.info()

In [ ]:
df1.describe(include='all')

In [ ]:
df1.head()

In [ ]:
df1 = df1.iloc[:,[5,4,7,0,1,3,6,2]]

In [ ]:
df1.head()

In [ ]:
df1.to_csv('House_data_cleaned.csv')

## Analyzing Data